In [1]:
from utils import *
import csv
from gensim.models import Word2Vec, KeyedVectors
import pickle
import numpy as np
import pandas as pd

import warnings
warnings.simplefilter('default')

In [2]:
#IMPORT MODELS AND DICTIONARIES (THIS FILES MUST BE CREATED)

reader = csv.reader(open('../dictionaries/dict_id_ntitle_final.csv', 'r', encoding='utf-8'))
dict_id_ntitle = {k:v for (k,v) in reader}
reader = csv.reader(open('../dictionaries/dict_id_ntitle_final.csv', 'r', encoding='utf-8'))
dict_ntitle_id = {v:k for (k,v) in reader}
reader = csv.reader(open('../dictionaries/dict_sorted_trackuri_id.csv', 'r')) #track_uri --- ID : csv to dict
dict_trackuri_id = {k:v for (k,v) in reader}
reader = csv.reader(open('../dictionaries/dict_sorted_trackuri_id.csv', 'r')) #track_uri --- ID : csv to dict
dict_id_trackuri = {v:k for (k,v) in reader} 

# MODEL TRACKS 
model_track = Word2Vec.load("../models/wv_model_mincount5_shuffle_size300_MPD")

# MODEL TITLES 
model_ntitle = Word2Vec.load("../models/wv_model_titles_MPD_final")

# UNCERTAINTY TITLES MODEL 
model_ntitle_uncert = KeyedVectors.load("../models/uncertainty/wv_model_certain_titles_MPD_final")
sigmas = pickle.load( open( "../models/uncertainty/sigmas.p", "rb" ) )

# most common 1000 tracks per title
title_tracks = np.load('../dictionaries/ntitles_1000tracks_final.npy').tolist()
artist_tracks = np.load('../dictionaries/nartists_1000tracks_final.npy').tolist()

model_artist = Word2Vec.load("../models/wv_model_artists_unique_size_100_MPD")
reader = csv.reader(open('../dictionaries/dict_artisturi_id.csv', 'r')) #track_uri --- ID : csv to dict
dict_artisturi_id = {k:v for (k,v) in reader}

/opt/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:5: ResourceWarning: unclosed file <_io.TextIOWrapper name='../dictionaries/dict_id_ntitle_final.csv' mode='r' encoding='utf-8'>
  """
/opt/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:7: ResourceWarning: unclosed file <_io.TextIOWrapper name='../dictionaries/dict_id_ntitle_final.csv' mode='r' encoding='utf-8'>
  import sys
/opt/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:9: ResourceWarning: unclosed file <_io.TextIOWrapper name='../dictionaries/dict_sorted_trackuri_id.csv' mode='r' encoding='ANSI_X3.4-1968'>
  if __name__ == '__main__':
/opt/anaconda/lib/python3.6/site-packages/gensim/utils.py:491: DeprecationWarning: Call to deprecated `cum_table` (Attribute will be removed in 4.0.0, use self.vocabulary.cum_table instead).
  setattr(self, attrib, None)
/opt/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:20: ResourceWarning: unclosed file <_io.BufferedReader name='../models/uncertai

## Recommender

In [3]:
## RECOMMENDATION BY TITLE ARTIST (TITLE == ARTIST)
def playlist_recommend_by_title_artist(playlist):
    weight_method = 1.0
    add_rec_artist = False
    if normalize_name(playlist["name"]) not in nop_titles and  normalize_name(playlist["name"]) in artist_tracks.keys():
        #check if artist is in the top 6.000 artists
        # Why 6.000? lets change it to 20.000
        if normalize_name(playlist["name"]) in list(artist_tracks.keys())[:6000]:
            tit = normalize_name(playlist["name"])
            #GET list of artist' songs
            rec_artist = [dict_trackuri_id[t[0]] for t in artist_tracks[tit] if t[0] in dict_trackuri_id ]
            if('tracks' in playlist):
                if(len(playlist['tracks'])==0):
                    add_rec_artist = True
                else:
                    artist_list = [item['artist_name'] for item in playlist['tracks']]
                    normalized_artist_list = [normalize_name(item) for item in artist_list]
                    if((len(set(artist_list))-1)/float(len(artist_list))<=0.2):
                        if(tit in normalized_artist_list):
                            add_rec_artist = True

    if(add_rec_artist):
        return rec_artist
    else:
        return []

In [4]:
# RECOMMENDATION BY TITLE
def playlist_recommend_by_title(playlist, title_tracks,params):
    title_recommendations = []
    tit = normalize_name(playlist["name"])   
        
    if tit not in dict_ntitle_id.keys():
        tit = tit.split(" ")[0]
    if tit not in dict_ntitle_id.keys():
        tit = tit.lower()
        tit = re.sub(r"[.,'\/#!$%\^\*;:{}=\_`~()@]", ' ', tit)
        tit = re.sub(r'\s+', ' ', tit).strip()      

     # ARTISTS 
    '''if "name" in playlist:
        if normalize_name(playlist["name"]) not in nop_titles and  normalize_name(playlist["name"]) in artist_tracks.keys():
            if normalize_name(playlist["name"]) in list(artist_tracks.keys())[:6000]:
                tit = normalize_name(playlist["name"])
                recommendations[playlist["pid"]] = [dict_trackuri_id[t[0]] for t in artist_tracks[tit] if t[0] in dict_trackuri_id ]    
    '''    
        
        
    if tit in dict_ntitle_id.keys(): 
        # find 10 most similar titles
        similar_titles = [tit] + [dict_id_ntitle[m[0]] for m in 
                      model_ntitle.wv.similar_by_word(dict_ntitle_id[tit], topn=params['TOP_TITLES'], restrict_vocab=None) ]
        similar_titles = [s for s in similar_titles if 
                      (model_ntitle.wv.similarity(dict_ntitle_id[tit],dict_ntitle_id[s])>params['TRESHOLD_TITLES'] and
                       sigmas[dict_ntitle_id[s]] < params['THRESHOLD_SIGMAS']
                      )]

        # take the most common tracks in these TOP_TITLES*1000 tracks 
        reco = {}
        for title in similar_titles:
            if title in title_tracks:
                for t in title_tracks[title]:
                    if t[0] in dict_trackuri_id: 
                        if dict_trackuri_id[t[0]] not in reco:
                            reco[dict_trackuri_id[t[0]]] = 0
                        reco[dict_trackuri_id[t[0]]] += t[1]*params['WEIGHT_MULTIPLIER']*model_ntitle.wv.similarity(dict_ntitle_id[tit],dict_ntitle_id[title])

        title_recommendations += [r[0] for r in sorted(reco.items(), key=lambda item: item[1] , reverse = True)[:750]]

        # if not enough tracks repeats with wider range
        if len(title_recommendations)<750:  
            similar_titles = [tit] + [dict_id_ntitle[m[0]] for m in model_ntitle.wv.similar_by_word(dict_ntitle_id[tit], topn=10, restrict_vocab=None) ]
  
            # take the most common tracks in these TOP_TITLES*1000 tracks 
            reco = {}
            for title in similar_titles:
                if title in title_tracks:
                    for t in title_tracks[title]:
                        if t[0] in dict_trackuri_id: 
                            if dict_trackuri_id[t[0]] not in reco:
                                reco[dict_trackuri_id[t[0]]] = 0
                            reco[dict_trackuri_id[t[0]]] += t[1]*params['WEIGHT_MULTIPLIER']*model_ntitle.wv.similarity(dict_ntitle_id[tit],dict_ntitle_id[title])

            title_recommendations += [r[0] for r in sorted(reco.items(), key=lambda item: item[1] , reverse = True)[:750]]
             # if not enough tracks repeats with wider range
            if len(title_recommendations)<500:  
                similar_titles = [tit] + [dict_id_ntitle[m[0]] for m in model_ntitle.wv.similar_by_word(dict_ntitle_id[tit], topn=20, restrict_vocab=None) ]

                # take the most common tracks in these TOP_TITLES*1000 tracks 
                reco = {}
                for title in similar_titles:
                    if title in title_tracks:
                        for t in title_tracks[title]:
                            if t[0] in dict_trackuri_id: 
                                if dict_trackuri_id[t[0]] not in reco:
                                    reco[dict_trackuri_id[t[0]]] = 0
                                reco[dict_trackuri_id[t[0]]] += t[1]*params['WEIGHT_MULTIPLIER']*model_ntitle.wv.similarity(dict_ntitle_id[tit],dict_ntitle_id[title])

                title_recommendations += [r[0] for r in sorted(reco.items(), key=lambda item: item[1] , reverse = True)[:750]]

            if len(title_recommendations)<750: # add popular songs if we have less than 750
                title_recommendations = title_recommendations + [str(i) for i in range(1000)]

    else:
        title_recommendations = title_recommendations + [str(i) for i in range(1000)]
    return title_recommendations

In [5]:
def recomendations_by_track_CENTROID(playlist,model_track,dict_trackuri_id,TOP):
    count = 0
    average = np.zeros(300)
    for t in playlist["tracks"]: 
        if t["track_uri"] in dict_trackuri_id:    
            if dict_trackuri_id[t["track_uri"]] in model_track.wv.vocab:
                average = np.add(average, model_track[dict_trackuri_id[t["track_uri"]]])
                count += 1
    average = average/count
    recommendations = [int(m[0]) for m in model_track.wv.similar_by_vector(average, topn=TOP, restrict_vocab=None)]
    return recommendations

In [6]:
def recsys_model(playlist, title_tracks,recommendations,params):   
    count = 0
    recommendations[playlist["pid"]]=[]
    
    # 1 - RECOMMENDATION BY TITLE == ARTIST 
    '''' ************* RECOMMENDATION BY ARTIST - ARTIST PLAYLIST: *************   '''
    if "name" in playlist: #check if the playlist has title
        rec_artist = playlist_recommend_by_title_artist(playlist)
        if(len(rec_artist)>0):
            recommendations[playlist["pid"]] = rec_artist
                         
    
    if playlist["num_samples"]==0: # 2 - RECOMMENDATION ONLY BY TITLE:
        recommendations[playlist["pid"]] += playlist_recommend_by_title(playlist, title_tracks,params)
    elif(playlist["num_samples"]==1):# RECOMMENDATION BY TRACKS  and TITLE 
        recommendations[playlist["pid"]] = recomendations_by_1_track(playlist,model_track,title_tracks,dict_trackuri_id,2)
    elif(playlist["num_samples"]==5):# ECOMMENDATION BY TRACKS  and TITLE
        recommendations[playlist["pid"]] = recomendations_by_5_track(playlist,model_track,title_tracks,dict_trackuri_id,4,1500,params)
    elif(playlist["num_samples"]==10):# ECOMMENDATION BY TRACKS  and TITLE
        recommendations[playlist["pid"]] = recomendations_by_5_track(playlist,model_track,title_tracks,dict_trackuri_id,16,5000,params)
    elif(playlist["num_samples"]==25):# ECOMMENDATION BY TRACKS  and TITLE
        recommendations[playlist["pid"]] = recomendations_by_5_track(playlist,model_track,title_tracks,dict_trackuri_id,16,5000,params)
    elif(playlist["num_samples"]==100):# ECOMMENDATION BY TRACKS  and TITLE
        recommendations[playlist["pid"]] = recomendations_by_5_track(playlist,model_track,title_tracks,dict_trackuri_id,16,10000,params)
    
    #just in case a small number of recommendations have been created
    recommendations[playlist["pid"]] += recommendations[playlist["pid"]] + [str(i) for i in range(500)]
        
    return recommendations 

In [7]:

def recomendations_by_5_track(playlist,model_track,title_tracks,dict_trackuri_id,FACTOR_TITLE=2,TOP=510,params):
    seed = playlist['tracks'][0]
    id_seed = dict_trackuri_id[seed['track_uri']]
    cont = TOP 
    
    rec_by_track= recomendations_by_track_CENTROID(playlist,model_track,dict_trackuri_id,TOP)
    
    r = {}
    for item in rec_by_track:
        r[str(item)]= cont
        cont-=1
    if "name" in playlist:
        rec_by_title = playlist_recommend_by_title(playlist, title_tracks,params)[:TOP]
        cont = TOP     
        for item in rec_by_title:
            if(item in r):
                r[item] = r[item] + cont/FACTOR_TITLE
            else:
                r[item]= cont/FACTOR_TITLE
            cont-=1

    sorted_list=sorted(r.items(), key=lambda x: x[1],reverse=True)
    recommendations = [item[0] for item in sorted_list]
    return recommendations

SyntaxError: non-default argument follows default argument (<ipython-input-7-a832f8bdb128>, line 2)

In [ ]:
def recomendations_by_1_track(playlist,model_track,title_tracks,dict_trackuri_id,FACTOR_TITLE =2):
    df2 = pd.read_csv('../../MPD/src//track_info2.csv',index_col='Unnamed: 0')
    TOP = 5000
    seed = playlist['tracks'][0]
    id_seed = dict_trackuri_id[seed['track_uri']]
    cont = TOP 
    if(int(id_seed)<590000):
        rec_by_track = model_track.wv.similar_by_vector(id_seed, topn=TOP, restrict_vocab=None)
    else:
        rec_by_track=[]
    rec_by_title = playlist_recommend_by_title(playlist, title_tracks)[:TOP]

    r = {}
    for item in rec_by_track:
        r[item[0]]= cont
        cont-=1
    cont = TOP     
    for item in rec_by_title:
        if(item in r):
            r[item] = r[item] + cont/FACTOR_TITLE
        else:
            r[item]= cont/FACTOR_TITLE
        cont-=1

    
    # add the 20 most populars songs for each of the similar artist to the seed
    if(int(id_seed)>=0):
        seed_artsist_id = dict_artisturi_id[playlist['tracks'][0]['artist_uri']]
        num_per_artist = 40
        if(int(seed_artsist_id)<194169): #artists with id higher than this are not considered since embedding are not computed
            # add top song of the seed artist:
            songs2add = df2[df2['artist_uri']==playlist['tracks'][0]['artist_uri']].sort_values(by='num_times',ascending=False)[:10].index
            songs2add = sorted([ int(dict_trackuri_id[s]) for s in sorted(songs2add)])[:num_per_artist]
            #add top songs of the similiar artist
            cont = num_per_artist     
            for item2 in songs2add:
                if(str(item2) in r):
                    r[str(item2)] = r[str(item2)] + cont*100
                else:
                    r[str(item2)]= cont*100
                cont = cont-1
                
            most_similar = model_artist.wv.most_similar(seed_artsist_id,topn=num_per_artist)
            for id_a in most_similar:
                songs2add = df2[df2['artist_uri']==dict_id_trackuri[id_a[0]]].sort_values(by='num_times',ascending=False)[:10].index
                songs2add = sorted([ int(dict_trackuri_id[s]) for s in sorted(songs2add)])[:10]
                #add top songs of the similiar artist
                cont = num_per_artist     
                for item2 in songs2add:
                    if(str(item2) in r):
                        r[str(item2)] = r[str(item2)] + cont*100
                    else:
                        r[str(item2)]= cont*100
                    cont = cont-1
    
    # rescore items according to the popularity
    for item in r:
        r[item] =r[item]* np.log(1+df2.loc[dict_id_trackuri[item]].num_times)
        
    sorted_list=sorted(r.items(), key=lambda x: x[1],reverse=True)
    recommendations = [item[0] for item in sorted_list]
    return recommendations

## Define Model Parameters

In [ ]:
#model parameters
params = {}
params['TOP'] = 800             
params['TOP_TITLES'] = 10
params['TRESHOLD_TITLES'] = 0.6
params['THRESHOLD_SIGMAS'] = 0.1
params['WEIGHT_MULTIPLIER'] = 2

def create_submission(recsys_model,name):
    
    process_challenge_set(recsys_model,name,dict_trackuri_id,dict_id_trackuri,title_tracks,params) 
    !python "../verify_submission.py" "../challenge_set.json" "../submissions/submission.csv"
    with open('../submissions/'+name+'.csv', 'rb') as f_in:
        with gzip.open('../submissions/'+name+'.csv.gz', 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)
            
            
name= "submission"
create_submission(recsys_model,name)